# Twitter - Baseline

### Pruebas con los distintos clasificadores para obtener los valores de transfer loss sin realizar adaptacion entre dominios

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *


#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

In [3]:
print tipo
print datasets[1]
print dims
print data_path

baseline
twitter
2000
data


# Pruebas con el dataset Twitter

In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
i = 1

for tgt in domains:
    print "Entrenando dominio %d de %d: %s" % (i, len(domains), tgt)
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    ruta_modelo = os.path.join(models_path, dataset_name, "indomain", "%s.pkl" % tgt)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(ruta_modelo, X_tr, y_tr)
    
    print "Score: %.3f" % svc.score(X_ts, y_ts)
    i = i+1
    
print "\nTarea terminada."

Entrenando dominio 1 de 3: rio2016
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.898
Entrenando dominio 2 de 3: thevoice
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.926
Entrenando dominio 3 de 3: general
Creando mejor modelo.


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

Guardando mejor modelo.
Score: 0.730

Tarea terminada.


In [6]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain",model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt sin adaptar
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s_%s.pkl" % (src, tgt)
            model_path = os.path.join(models_path, dataset_name, tipo, model_name)
            
            svc2 = load_best_score(model_path, X_tr, y_tr)
            t_error = 1-svc2.score(X_ts, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['Baseline',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "\nPruebas completadas.\n"

Tarea 1 de 6
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 2 de 6
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 3 de 6
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 4 de 6
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 5 de 6
Cargando modelo existente.
Creando mejor modelo.


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

Guardando mejor modelo.
Tarea 6 de 6
Cargando modelo existente.
Creando mejor modelo.


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

Guardando mejor modelo.

Pruebas completadas.



In [7]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,Baseline,r->t,rio2016,thevoice,7.362689,20.845170,13.482481
2,Baseline,r->g,rio2016,general,26.998127,43.116491,16.118364
3,Baseline,t->r,thevoice,rio2016,10.208711,13.747731,3.539020
4,Baseline,t->g,thevoice,general,26.998127,39.874641,12.876514
5,Baseline,g->r,general,rio2016,10.208711,24.183303,13.974592
6,Baseline,g->t,general,thevoice,7.362689,23.236269,15.873580


In [9]:
new_scores_path1 = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
new_scores_path2 = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path1
print "Guardando en %s" % new_scores_path2

df.to_csv(new_scores_path1, columns=df.columns)
df.to_csv(new_scores_path2, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/baseline/me1_2000.csv
Guardando en scores/twitter/baseline/me2_2000.csv
Resultados guardados.
